In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup, Tag
from urllib.parse import urljoin
import pandas as pd
import json
import time
import random

In [2]:
df = pd.read_csv("/kaggle/input/all-pokemon-names-gen-9/Pokemon_names.csv")

In [3]:
df

,pokedex_id,Pokemon,url,gen
0,#0001,Bulbasaur,/wiki/Bulbasaur_(Pok%C3%A9mon),1
1,#0002,Ivysaur,/wiki/Ivysaur_(Pok%C3%A9mon),1
2,#0003,Venusaur,/wiki/Venusaur_(Pok%C3%A9mon),1
3,#0004,Charmander,/wiki/Charmander_(Pok%C3%A9mon),1
4,#0005,Charmeleon,/wiki/Charmeleon_(Pok%C3%A9mon),1
...,...,...,...,...
1020,#1021,Raging Bolt,/wiki/Raging_Bolt_(Pok%C3%A9mon),9
1021,#1022,Iron Boulder,/wiki/Iron_Boulder_(Pok%C3%A9mon),9
1022,#1023,Iron Crown,/wiki/Iron_Crown_(Pok%C3%A9mon),9
1023,#1024,Terapagos,/wiki/Terapagos_(Pok%C3%A9mon),9


In [4]:
def get_soup(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        print(f"Error fetching page: {e}")
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup

In [5]:
pokemon_names = df['Pokemon'].to_list()
urls = df['url'].to_list()

In [6]:
base_url="https://bulbapedia.bulbagarden.net"

In [32]:
pokemon_types = {
    "Normal", "Fire", "Water", "Grass", "Electric", "Ice", "Fighting", "Poison",
    "Ground", "Flying", "Psychic", "Bug", "Rock", "Ghost", "Dragon", "Dark",
    "Steel", "Fairy"
}

In [35]:
i = 1011
full_url = base_url + urls[i]
def get_typing(url, pokemon_types):
    soup = get_soup(full_url)
    type_tag = soup.find_all('b', string=lambda text: text and 'Type' in text)[0]
    next_table = type_tag.find_next('table', class_='roundy').find("tbody").find("tr")
    types = next_table.find_all('b', string=lambda text: text in pokemon_types)
    return [tag.get_text() for tag in types]
get_typing(full_url, pokemon_types)

['Grass', 'Ghost']

In [36]:
type_dict = dict()
for i in range(len(urls)):
    full_url = base_url + urls[i]
    type_dict[pokemon_names[i]] = get_typing(full_url, pokemon_types)

In [37]:
names = []
types = []
for name, type_of in type_dict.items():
    names.append(name)
    types.append(type_of)

In [38]:
df_types = pd.DataFrame({"Pokemon_Name": names, "Types": types})

In [39]:
df_types.to_csv("poke_types.csv", index=False)

In [40]:
df_types

,Pokemon_Name,Types
0,Bulbasaur,"[Grass, Poison]"
1,Ivysaur,"[Grass, Poison]"
2,Venusaur,"[Grass, Poison]"
3,Charmander,[Fire]
4,Charmeleon,[Fire]
...,...,...
1020,Raging Bolt,"[Electric, Dragon]"
1021,Iron Boulder,"[Rock, Psychic]"
1022,Iron Crown,"[Steel, Psychic]"
1023,Terapagos,[Normal]
